# Implement IWRU-net for Watermark Removal

In [2]:
import torch
import torch.nn as nn

## Define the architecture

In [ ]:
class ConvRelu(nn.Module):
    def __init__(self, channels_in, channels_out, kernel_size=3, padding=1, bias=False):
        super(ConvRelu, self).__init__()

        self.conv_relu = nn.Sequential(
            nn.Conv2d(
                channels_in,
                channels_out,
                kernel_size=kernel_size,
                padding=padding,
                bias=bias,
            ),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv_relu(x)


In [ ]:
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

    def forward(self, x):
        pass